# Crag level granularity? Maybe some other time.  

In [1]:
# Trying to get hyperlinks for all states so we can run the scraping script on all states.

# Imports and get the html data from MountainProject. 
import pandas as pd
import numpy as np
import string
import requests
from bs4 import BeautifulSoup

url = 'https://www.mountainproject.com/route-guide'
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

# Put all hyperlinks on the page into a list.
data = []
for link in soup.find_all('a', class_= 'text-truncate float-xs-left'):
    data.append(link.get('href'))
    
# List of US States that we'll use to remove unwanted hyperlinks.
# Mississippi and Nebrasaka don't have pages, which makes sense,
# considering how flat they are.
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","New-Hampshire","New-Jersey","New-Mexico","New-York",
  "North-Carolina","North-Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "Rhode-Island","South-Carolina","South-Dakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","West-Virginia","Wisconsin","Wyoming"]

# List of US States and their abbreviations for a dictionary.
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New-Hampshire': 'NH',
    'New-Jersey': 'NJ',
    'New-Mexico': 'NM',
    'New-York': 'NY',
    'North-Carolina': 'NC',
    'North-Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode-Island': 'RI',
    'South-Carolina': 'SC',
    'South-Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West-Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# Make the states lowercase to match hyperlinks. 
states = [x.lower() for x in states]
us_state_abbrev = dict((k.lower(),v) for k,v in us_state_abbrev.items())

# Loop through each hyperlink and see if it contains a state as a subsrting.
# If it does, add the link to a list containing links for each state page. 
US = []
for datum in data:
    for state in states:
        if state in datum:
            US.append([state,datum])
        
# Remove duplicate links, Vermont Ice+Mixed, and Tennessee Wall (they are duplicates!)
# Check length. Should be 48. 
res = [] 
[res.append(x) for x in US if x not in res] 
# US=list(set(US))
res.remove(['tennessee','https://www.mountainproject.com/area/105851828/the-tennessee-wall'])
res.remove(['vermont','https://www.mountainproject.com/area/107280521/vermont-ice-and-mixed'])
links = dict(res)

In [25]:
def climb_dataframe(link, state):
    url = link
    page = requests.get(link)
    soup = BeautifulSoup(page.content,'html.parser')
    #dfClimb = pd.DataFrame()
    route_count = []
    route_type = []
    for climb_type in climb_types:
        for route in soup.find_all('span', class_ = "lef-nav-" + climb_type):
            route_count.append(route.text)
            route_type.append(climb_type)

    route_count = [s.strip() for s in route_count]
    route_count = [s.replace(' /','') for s in route_count]
    route_count = [s.replace(',','') for s in route_count]
    route_count = [int(s) for s in route_count]
    df = pd.DataFrame(data = route_count)
    df['type'] = route_type
    df = df.set_index('type')
    df = df.sum(level = 'type')
    #df.assign(ST = us_state_abbrev[state])
    df['ST'] = us_state_abbrev[state]
    return df

In [26]:
climb_types = ['Trad','Sport','Toprope','Boulder','Ice','Aid','Mixed','Alpine']
climb_df = pd.DataFrame()
for sta, link in links.items():
    climb_df = climb_df.append(climb_dataframe(link,sta))

In [27]:
climb_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 384 entries, Trad to Alpine
Data columns (total 2 columns):
0     384 non-null int64
ST    384 non-null object
dtypes: int64(1), object(1)
memory usage: 9.0+ KB


In [19]:
us_state_abbrev[state]

'WY'

In [29]:
climb_df.head(50)

,0,ST
type,,
Trad,249,AL
Sport,317,AL
Toprope,103,AL
Boulder,873,AL
Ice,0,AL
Aid,1,AL
Mixed,0,AL
Alpine,0,AL
Trad,807,AK


In [34]:
def test_climb_dataframe(link, state):
    url = link
    page = requests.get(link)
    soup = BeautifulSoup(page.content,'html.parser')
    #dfClimb = pd.DataFrame()
    route_count = []
    route_type = []
    for climb_type in climb_types:
        for route in soup.find_all('span', class_ = "lef-nav-" + climb_type):
            route_count.append(route.text)
            route_type.append(climb_type)

    route_count = [s.strip() for s in route_count]
    route_count = [s.replace(' /','') for s in route_count]
    route_count = [s.replace(',','') for s in route_count]
    route_count = [int(s) for s in route_count]
    df = pd.DataFrame(data = route_count)
    df['type'] = route_type
    #df = df.set_index('type')
    #df = df.sum(level = 'type')
    #df.assign(ST = us_state_abbrev[state])
    df['ST'] = us_state_abbrev[state]
    return df

In [35]:
climb_types = ['Trad','Sport','Toprope','Boulder','Ice','Aid','Mixed','Alpine']
test_df = pd.DataFrame()
for sta, link in links.items():
    test_df = test_df.append(test_climb_dataframe(link,sta))

In [37]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9120 entries, 0 to 255
Data columns (total 3 columns):
0       9120 non-null int64
type    9120 non-null object
ST      9120 non-null object
dtypes: int64(1), object(2)
memory usage: 285.0+ KB


In [38]:
test_df.head()

,0,type,ST
0,1,Trad,AL
1,7,Trad,AL
2,0,Trad,AL
3,10,Trad,AL
4,0,Trad,AL


In [39]:
len(test_df['type'])

9120

In [82]:
hope = test_df
hope.columns = ['num','type','ST']
ind_values = []
ind_values = [ind_values.append(x) for x in range(0,len(hope))]
hope.index.name = 'area'
hope.head()

,num,type,ST
area,,,
0,1,Trad,AL
1,7,Trad,AL
2,0,Trad,AL
3,10,Trad,AL
4,0,Trad,AL


In [84]:
len(hope)

9120

236

In [74]:
hope.head()

,num,type,ST
area,,,
0,1,Trad,AL
1,7,Trad,AL
2,0,Trad,AL
3,10,Trad,AL
4,0,Trad,AL


In [76]:
#table = pd.pivot_table(hope,values = 'num', columns = 'type', index ='ST', aggfunc = np.sum)
table = pd.pivot_table(hope,values = 'num', columns = 'type',index = 'area', aggfunc = np.sum)

In [77]:
table

type,Aid,Alpine,Boulder,Ice,Mixed,Sport,Toprope,Trad
area,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5564.0
1,NaN,NaN,NaN,NaN,NaN,0.0,NaN,5284.0
2,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2829.0
3,NaN,NaN,1.0,NaN,NaN,1473.0,NaN,3545.0
4,NaN,NaN,NaN,0.0,NaN,2670.0,76.0,5919.0
5,0.0,NaN,NaN,NaN,NaN,2603.0,178.0,3549.0
6,NaN,NaN,60.0,NaN,0.0,356.0,270.0,941.0
7,NaN,0.0,175.0,NaN,NaN,823.0,263.0,779.0
8,NaN,NaN,NaN,0.0,NaN,755.0,276.0,667.0


In [67]:
table['Aid']['num']

1535

In [71]:
hope.index.name = 'area'

,num,type,ST
area,,,
0,1,Trad,AL
1,7,Trad,AL
2,0,Trad,AL
3,10,Trad,AL
4,0,Trad,AL
